In [55]:
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./dracula.mplstyle')
import logging
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)
LOG.info('Hello from notebook')

from pydataset import data

import frankenfit as ff
reload(ff.core)
reload(ff.transforms)
reload(ff.io)
reload(ff.graph)
reload(ff)

import pyarrow.dataset as ds

logging.getLogger('frankenfit').setLevel(logging.INFO)

In [56]:
ff.Assign(foo=1, bar=2, logprice=lambda df: np.log1p(df['price']), tag='boo')

Assign(tag='boo', assignments={'foo': 1, 'bar': 2, 'logprice': <function <lambda> at 0x7f6d5b40acb0>})

In [65]:
df = data('diamonds').reset_index().set_index('index')

(
    ff.Pipeline()
    .assign(
        intercept=1,
        grp=lambda df: df.index % 5,
        grp_2=lambda self, df: df.index % self.bindings()['k']
    )
).apply(df, bindings={'k': 3})

,carat,cut,color,clarity,depth,table,price,x,y,z,intercept,grp,grp_2
index,,,,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,1,1,1
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1,2,2
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,1,3,0
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1,4,1
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50,1,1,2
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61,1,2,0
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56,1,3,1


In [66]:
pip = (
    ff.Pipeline()
    .assign({
        'intercept': 1,
        ff.HPFmtStr('grp_{k}'): lambda self, df: df.index % self.bindings()['k']
    }, tag='foo')
)
display(pip)
display(pip.hyperparams())
display(pip.apply(df, bindings={'k': 3}).head(10))

Pipeline(tag='Pipeline#20', transforms=[Assign(tag='foo', assignments=HPDict(mapping={'intercept': 1, HPFmtStr(name='grp_{k}'): <function <lambda> at 0x7f6d5b15f400>}, name=None))])

{'k'}

,carat,cut,color,clarity,depth,table,price,x,y,z,intercept,grp_3
index,,,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,1,1
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1,2
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,1,0
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1,1
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,1,2
6,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48,1,0
7,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47,1,1
8,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53,1,2
9,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49,1,0
